In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rf').getOrCreate()

In [3]:
# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("libsvm").load("/FileStore/tables/sample_libsvm_data.txt")

In [4]:
data.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows

In [5]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Decision Tree Example

This is just a quick walkthrough of the Documentation's Example of Decision Tree:

In [7]:
from pyspark.ml.classification import DecisionTreeClassifier

In [8]:
# Train a Decision Tree model.
dt = DecisionTreeClassifier()

# Train model.  This also runs the indexers.
model = dt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.show(5)

+-----+--------------------+-------------+-----------+----------+
label| features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
 0.0|(692,[100,101,102...| [0.0,40.0]| [0.0,1.0]| 1.0|
 0.0|(692,[121,122,123...| [27.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[124,125,126...| [27.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[124,125,126...| [27.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[126,127,128...| [27.0,0.0]| [1.0,0.0]| 0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 5 rows

In [9]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.121212

# Random Forest Example

This is just a quick walkthrough of the Documentation's Example of Random Forest:

In [11]:
from pyspark.ml.classification import RandomForestClassifier

In [12]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=150)

# Train model. This also runs the indexers.
model = rf.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.show(5)

+-----+--------------------+-------------+--------------------+----------+
label| features|rawPrediction| probability|prediction|
+-----+--------------------+-------------+--------------------+----------+
 0.0|(692,[100,101,102...| [64.0,86.0]|[0.42666666666666...| 1.0|
 0.0|(692,[121,122,123...| [147.0,3.0]| [0.98,0.02]| 0.0|
 0.0|(692,[124,125,126...| [149.0,1.0]|[0.99333333333333...| 0.0|
 0.0|(692,[124,125,126...| [125.0,25.0]|[0.83333333333333...| 0.0|
 0.0|(692,[126,127,128...| [139.0,11.0]|[0.92666666666666...| 0.0|
+-----+--------------------+-------------+--------------------+----------+
only showing top 5 rows

In [13]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.030303

In [14]:
# Variable importance

# Not a very good example to show this!
model.featureImportances

Out[86]: SparseVector(692, {97: 0.0009, 100: 0.0008, 129: 0.0004, 148: 0.0008, 152: 0.0002, 154: 0.0002, 159: 0.0001, 181: 0.0005, 184: 0.0003, 186: 0.0008, 204: 0.0003, 207: 0.0004, 208: 0.0037, 216: 0.0049, 236: 0.0003, 237: 0.0002, 238: 0.0006, 242: 0.0003, 243: 0.005, 244: 0.0062, 263: 0.0011, 264: 0.0038, 272: 0.0047, 273: 0.0069, 274: 0.0042, 287: 0.0008, 288: 0.0024, 290: 0.0053, 294: 0.0004, 295: 0.0004, 296: 0.0002, 300: 0.0126, 301: 0.0004, 302: 0.0108, 304: 0.0004, 315: 0.0016, 317: 0.0172, 320: 0.0015, 322: 0.0039, 323: 0.0065, 326: 0.0002, 327: 0.0004, 328: 0.0105, 329: 0.0013, 330: 0.0139, 331: 0.0012, 342: 0.0009, 344: 0.0131, 346: 0.0038, 347: 0.0011, 350: 0.0187, 351: 0.0125, 354: 0.0012, 355: 0.0008, 356: 0.0057, 357: 0.0073, 358: 0.012, 370: 0.0005, 371: 0.0122, 372: 0.0559, 373: 0.0015, 377: 0.0033, 378: 0.0199, 379: 0.0112, 380: 0.0004, 385: 0.012, 386: 0.0054, 397: 0.0012, 398: 0.0004, 399: 0.0117, 400: 0.0056, 401: 0.0055, 402: 0.0012, 405: 0.0247, 406: 0.0039, 407: 0.0198, 408: 0.0011, 409: 0.0008, 414: 0.0214, 415: 0.0073, 416: 0.0004, 425: 0.0013, 426: 0.0067, 427: 0.0204, 428: 0.0152, 429: 0.0052, 433: 0.0099, 434: 0.0528, 435: 0.0248, 436: 0.0013, 437: 0.0011, 440: 0.0006, 441: 0.0071, 442: 0.0059, 443: 0.0004, 444: 0.0009, 454: 0.0117, 455: 0.0209, 456: 0.008, 457: 0.0054, 460: 0.0012, 461: 0.014, 462: 0.0286, 465: 0.0007, 466: 0.0004, 468: 0.0243, 470: 0.0127, 471: 0.0014, 472: 0.0018, 483: 0.0346, 484: 0.0218, 489: 0.0251, 490: 0.0329, 493: 0.0006, 494: 0.0005, 495: 0.0003, 496: 0.0083, 497: 0.0003, 499: 0.0009, 510: 0.0059, 511: 0.0364, 512: 0.025, 516: 0.0007, 517: 0.0126, 518: 0.0004, 523: 0.0016, 524: 0.0041, 538: 0.0004, 539: 0.0073, 540: 0.0251, 545: 0.0014, 546: 0.0003, 548: 0.0004, 552: 0.0049, 566: 0.0004, 568: 0.0025, 569: 0.0044, 570: 0.001, 576: 0.0003, 595: 0.0004, 601: 0.0004, 606: 0.0006, 607: 0.0036, 624: 0.0005, 628: 0.0012, 629: 0.0014, 634: 0.0011, 658: 0.0005, 666: 0.0005, 680: 0.0003, 682: 0.0007})

## Gradient Boosted Trees

Gradient-boosted trees (GBTs) are a popular classification and regression method using ensembles of decision trees. More information about the spark.ml implementation can be found further in the section on [GBTs.](http://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-trees-gbts). For more information on the algorithm itself, please see the [spark.mllib documentation on GBTs.](http://spark.apache.org/docs/latest/mllib-ensembles.html#gradient-boosted-trees-gbts)

Luckily Spark makes very easy to use, basically just an import switch:

In [16]:
from pyspark.ml.classification import GBTClassifier

In [17]:
# Train a GBT model.
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

# Train model.  This also runs the indexers.
model = gbt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 1.0| 0.0|(692,[100,101,102...|
 0.0| 0.0|(692,[121,122,123...|
 0.0| 0.0|(692,[124,125,126...|
 0.0| 0.0|(692,[124,125,126...|
 0.0| 0.0|(692,[126,127,128...|
+----------+-----+--------------------+
only showing top 5 rows

In [18]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.121212

So this data isn't really realistic enough to really judge to effectiveness of GBT models, this data makes it seem like they are perfection, instead of just an improvement on normal Random Forests.

Let's move on to a more realistic example!